In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [ ]:
TOKEN = 'токен от мни приложения ВК'
DOMAIN = 'id профиля 1'
DOMAIN2 = 'id профиял 2'


In [23]:
'htpps://id.vk.vom/oauth2/auth'

'htpps://id.vk.vom/oauth2/auth'

In [24]:
url="https://api.vk.com/method/groups.get"
req = requests.post(url, params={
    'access_token': TOKEN,
    'user_id': DOMAIN,
    'v': '5.199'
})

In [25]:
print(req)

<Response [200]>


In [26]:
req.text

'{"response":{"count":296,"items":[22822305,57846937,147845620,57876954,23064236,157369801,30602036,9159189,22522055,78616012,101982925,32370614,150802579,84600270,51812607,31333866,71822322,9580285,8458649,71474813,170528132,52599708,212496568,25901021,16775977,10362317,36508616,186112991,150618202,31769636,156197485,33118207,66814271,88165975,49033185,159848117,39488246,60708045,23977452,13643401,144962203,58662334,169194674,179997490,48392922,182719373,16647180,53484080,166884737,42793953,113866673,14000591,4070797,11445015,129078557,61166138,63632654,25280329,152683782,64250751,48864,84564383,147793006,32151480,141632613,35540891,188353599,3551694,138723788,75225221,75331219,182618011,16123937,11204681,183688289,127451122,58767484,30651595,159800197,92440166,24011636,110715182,17700656,40155990,29218811,112199313,101746647,132275903,49355363,27527249,54890300,9802860,102862830,79997904,29725517,53916844,152950619,69593870,145058713,118441552,46337218,36542160,38629337,162420900,786

In [27]:
url2="https://api.vk.com/method/wall.get"
req2 = requests.post(url2, params={
    'access_token': TOKEN,
    'domain': DOMAIN,
    'v': '5.199'
})

In [28]:
req2.text

'{"response":{"count":157,"items":[{"inner_type":"wall_wallpost","copy_history":[{"inner_type":"wall_wallpost","type":"post","attachments":[{"type":"photo","photo":{"album_id":-7,"date":1750184541,"id":457636557,"owner_id":-38901360,"access_key":"be0c241f0998398682","post_id":2859527,"sizes":[{"height":96,"type":"s","width":72,"url":"https:\\/\\/sun9-26.userapi.com\\/s\\/v1\\/ig2\\/YNhLSF9m1ceoSYuZaDt99PHWDkTkWsvWEjWOGYMvccaCGBl0I9gG_jUHqg8hfbM_yBFPNM1MRF3tIvC5Z8xyCZ8l.jpg?quality=95&as=32x43,48x64,72x96,108x144,160x213,240x320,360x480,480x640,540x720,640x853,720x960,1080x1440,1280x1707,1440x1920,1920x2560&from=bu&cs=72x0"},{"height":213,"type":"m","width":160,"url":"https:\\/\\/sun9-26.userapi.com\\/s\\/v1\\/ig2\\/YNhLSF9m1ceoSYuZaDt99PHWDkTkWsvWEjWOGYMvccaCGBl0I9gG_jUHqg8hfbM_yBFPNM1MRF3tIvC5Z8xyCZ8l.jpg?quality=95&as=32x43,48x64,72x96,108x144,160x213,240x320,360x480,480x640,540x720,640x853,720x960,1080x1440,1280x1707,1440x1920,1920x2560&from=bu&cs=160x0"},{"height":853,"type":"x","w

In [29]:
url3="https://api.vk.com/method/wall.get"
req2 = requests.post(url2, params={
    'access_token': TOKEN,
    'domain': DOMAIN,
    'v': '5.199'
})

In [30]:
data = req.json()
group_ids = data['response']['items']

In [31]:
if 'error' in data:
    print(f"Ошибка API: {data['error']['error_msg']}")
    group_ids = []
else:
    group_ids = data['response']['items']
    print(f"Найдено групп: {len(group_ids)}")

# Если есть группы, получаем их названия
if group_ids:
    # Ограничим количество для теста (можно убрать)
    group_ids_sample = group_ids[:50]  # Первые 50 групп
    
    # Преобразуем ID в строки и фильтруем некорректные значения
    valid_group_ids = [str(gid) for gid in group_ids_sample if gid is not None]
    
    if not valid_group_ids:
        print("Нет корректных ID групп для запроса")
        group_names = []
    else:
        url_by_id = "https://api.vk.com/method/groups.getById"
        req_names = requests.post(url_by_id, params={
            'access_token': TOKEN,
            'group_ids': ','.join(valid_group_ids),
            'v': '5.199',
            'fields': 'name'  # Явно указываем, что нужны названия
        })
        
        # Парсим ответ
        group_data = req_names.json()
        print("Ответ от groups.getById:", group_data)  # Для отладки
        
        # Проверяем наличие ошибок
        if 'error' in group_data:
            print(f"Ошибка при получении названий: {group_data['error']['error_msg']}")
            group_names = []
        elif 'response' in group_data:
            # Безопасная проверка типа данных
            response_data = group_data['response']
            
            if isinstance(response_data, list):
                # Обычный случай - список групп
                group_names = [group['name'] for group in response_data if isinstance(group, dict) and 'name' in group]
                
                # Выводим список названий групп
                print("\nНазвания групп:")
                print("-" * 50)
                for i, name in enumerate(group_names, 1):
                    print(f"{i}. {name}")
                    
            elif isinstance(response_data, dict):
                # Если вернулся один объект группы
                if 'name' in response_data:
                    group_names = [response_data['name']]
                    print(f"\nНазвание группы: {response_data['name']}")
                else:
                    print("Неожиданный формат объекта группы")
                    group_names = []
                    
            else:
                print(f"Неожиданный тип ответа: {type(response_data)}")
                print(f"Содержимое ответа: {response_data}")
                group_names = []
                
        else:
            print("Неожиданный формат ответа")
            print(f"Полный ответ: {group_data}")
            group_names = []

Найдено групп: 296
Ответ от groups.getById: {'response': {'groups': [{'id': 22822305, 'name': 'ВКонтакте', 'screen_name': 'vk', 'is_closed': 0, 'type': 'page', 'photo_50': 'https://sun87-1.userapi.com/s/v1/ig2/HHLuuf1w_841-Sz051TU6CsfNYhDkstlEoKt6kRWPIve9TS6DGVWkbgr4dWDEgOWYm2IMZkErqr8825jauMHF_ss.jpg?quality=95&crop=0,0,1000,1000&as=32x32,48x48,72x72,108x108,160x160,240x240,360x360,480x480,540x540,640x640,720x720&ava=1&cs=50x50', 'photo_100': 'https://sun87-1.userapi.com/s/v1/ig2/HHLuuf1w_841-Sz051TU6CsfNYhDkstlEoKt6kRWPIve9TS6DGVWkbgr4dWDEgOWYm2IMZkErqr8825jauMHF_ss.jpg?quality=95&crop=0,0,1000,1000&as=32x32,48x48,72x72,108x108,160x160,240x240,360x360,480x480,540x540,640x640,720x720&ava=1&cs=100x100', 'photo_200': 'https://sun87-1.userapi.com/s/v1/ig2/HHLuuf1w_841-Sz051TU6CsfNYhDkstlEoKt6kRWPIve9TS6DGVWkbgr4dWDEgOWYm2IMZkErqr8825jauMHF_ss.jpg?quality=95&crop=0,0,1000,1000&as=32x32,48x48,72x72,108x108,160x160,240x240,360x360,480x480,540x540,640x640,720x720&ava=1&cs=200x200'}, {'id': 5

In [32]:
url_by_id = "https://api.vk.com/method/groups.getId"
req_names = requests.post(url_by_id, params={
    'access_token': TOKEN,
    'group_ids': ','.join(map(str, group_ids)),
    'v': '5.199'
})

In [34]:
group_data = req_names.json()
print("Полный ответ от API:", group_data)  # Для диагностики

# Сначала проверяем наличие ошибки
if 'error' in group_data:
    error_msg = group_data['error'].get('error_msg', 'Неизвестная ошибка')
    error_code = group_data['error'].get('error_code', 'Неизвестный код')
    print(f"Ошибка API (код {error_code}): {error_msg}")
    group_names = []
# Затем проверяем наличие response
elif 'response' in group_data:
    # Проверяем, что response является списком
    if isinstance(group_data['response'], list):
        group_names = [group['name'] for group in group_data['response']]
        
        # Выводим список названий групп
        print("\nНазвания групп:")
        print("-" * 50)
        for i, name in enumerate(group_names, 1):
            print(f"{i}. {name}")
    else:
        print(f"Неожиданный формат response: {type(group_data['response'])}")
        print(f"Содержимое: {group_data['response']}")
        group_names = []
else:
    print("Неожиданный формат ответа от API")
    print(f"Полный ответ: {group_data}")
    group_names = []

Полный ответ от API: {'error': {'error_code': 3, 'error_msg': 'Unknown method passed', 'request_params': [{'key': 'group_ids', 'value': '22822305,57846937,147845620,57876954,23064236,157369801,30602036,9159189,22522055,78616012,101982925,32370614,150802579,84600270,51812607,31333866,71822322,9580285,8458649,71474813,170528132,52599708,212496568,25901021,16775977,10362317,36508616,186112991,150618202,31769636,156197485,33118207,66814271,88165975,49033185,159848117,39488246,60708045,23977452,13643401,144962203,58662334,169194674,179997490,48392922,182719373,16647180,53484080,166884737,42793953,113866673,14000591,4070797,11445015,129078557,61166138,63632654,25280329,152683782,64250751,48864,84564383,147793006,32151480,141632613,35540891,188353599,3551694,138723788,75225221,75331219,182618011,16123937,11204681,183688289,127451122,58767484,30651595,159800197,92440166,24011636,110715182,17700656,40155990,29218811,112199313,101746647,132275903,49355363,27527249,54890300,9802860,102862830,7999